In [4]:
import pandas as pd

# loding vocab from pickle file
import pickle
vocab = pickle.load(open('vocab.pkl', 'rb'))
# convert vocab to dataframe
vocab = pd.DataFrame(vocab, columns=['word', 'count'])
# filter words that occur less than 2000 times
vocab = vocab[vocab['count'] > 2000]

# make dataframe where each column is the value in each row of the vocab dataframe word column
bow_columns = pd.DataFrame(columns=vocab['word'].values)

In [5]:
chunk = pd.read_csv('tokenized_data.csv', nrows=10)

In [25]:
# make set of all characters in the vocab words
all_char = set(char for char in vocab['word'].to_string())
all_char

{'\n',
 ' ',
 "'",
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '=',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '~',
 '©',
 '®',
 '™',
 '▪',
 '♦',
 '️',
 '🙂'}

In [1]:
import re
pattern = re.compile(r'(?:\'|")([a-z0-9"\.+-/_=:`|~©™®▪♦🙂 ]*)(?:\'|"),')

In [4]:
# convert to parquet
# load data with dask from csv
from dask import dataframe as dd
from dask.diagnostics import ProgressBar
df = dd.read_csv('fake_news_cleaned_filtered.csv')
# parse the content column
df = df.repartition(partition_size=1000)
df['0'] = df['0'].map_partitions(lambda x: x.apply(lambda y: pattern.findall(y)))
# save to parquet
with ProgressBar():
    df.to_parquet('fake_news_cleaned_filtered.parquet', index=False)

In [36]:
# read start of csv
import pandas as pd
df = pd.read_csv('fake_news_cleaned_filtered.csv', nrows=10)
df

,0
0,"['wall', 'street', 'journal', 'excellent', 'ar..."
1,"['unfortunately', ""n't"", 'yet', 'attacked', 'i..."
2,"['los', 'angeles', 'police', 'department', 'de..."
3,"['white', 'house', 'decided', 'quietly', 'with..."
4,"['``', 'time', 'come', 'cut', 'tongue', 'suppo..."
5,"['central', 'american', 'nation', 'six', 'stat..."
6,"['un-backed', 'police', 'massacred', 'haitian'..."
7,"['come', 'surprise', '``', 'rebel', ""''"", 'u',..."
8,"['``', 'police', 'finally', 'left', 'campus', ..."
9,"['zambia', 'must', 'clarify', 'whether', 'host..."


In [27]:
# load in chunk of data
chunk = pd.read_csv('tokenized_data.csv', nrows=10000)

# for each row in the chunk
for index, row in chunk.iterrows():
    # add 10000 rows of zeros to the bow dataframe
    zeros = pd.DataFrame(0, index=range(10000), columns=bow_columns.columns)
    bow = bow_columns.concat()

    # split the content into a list of words
    words = pattern.findall(row['content'])
    # make new row of zeros in the bow dataframe using concat
    bow = bow.append(pd.Series(0, index=bow.columns), ignore_index=True)
    # for each word in the list of words in the row add 1 to the corresponding column in the bow dataframe
    for word in words:
        if word in bow.columns:
            bow.loc[index, word] += 1


C:\Users\musta\AppData\Local\Temp\ipykernel_30988\2901767100.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bow = bow.append(pd.Series(0, index=bow.columns), ignore_index=True)
C:\Users\musta\AppData\Local\Temp\ipykernel_30988\2901767100.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bow = bow.append(pd.Series(0, index=bow.columns), ignore_index=True)
C:\Users\musta\AppData\Local\Temp\ipykernel_30988\2901767100.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bow = bow.append(pd.Series(0, index=bow.columns), ignore_index=True)
C:\Users\musta\AppData\Local\Temp\ipykernel_30988\2901767100.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

KeyboardInterrupt: 